<a href="https://colab.research.google.com/github/advapplab/esgBERT_hf/blob/main/esgBERT_Finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install transformers
!pip install huggingface_hub
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
! pip install essential-generators

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 01 Load Dataset

In [55]:

! wget -nc https://raw.githubusercontent.com/advapplab/esgBERT_hf/main/data/carbon_emissions.csv
! wget -nc https://raw.githubusercontent.com/advapplab/esgBERT_hf/main/data/community_relations.csv

! wget -nc https://raw.githubusercontent.com/Gunjitbedi/Text-Classification/master/corpus.csv

File ‘carbon_emissions.csv’ already there; not retrieving.

File ‘community_relations.csv’ already there; not retrieving.

File ‘corpus.csv’ already there; not retrieving.



In [56]:
import pandas as pd

training_pd = pd.DataFrame()

ce_pd = pd.read_csv('/content/carbon_emissions.csv', header='infer')
cr_pd = pd.read_csv('/content/community_relations.csv', header='infer')

corpus_pd = pd.read_csv('corpus.csv', header='infer', encoding='latin-1').tail(1500)

header = ['Sentence']

ce_pd = ce_pd[header].dropna()
cr_pd = cr_pd[header].dropna()

ce_pd['label'] = 1
cr_pd['label'] = 0
corpus_pd['label'] = 2

corpus_pd.rename(columns = {'text':'Sentence'}, inplace = True)

training_pd = ce_pd.append(cr_pd)
training_pd = training_pd.reset_index()
training_pd = training_pd.append(corpus_pd).reset_index().drop(['level_0', 'index'], axis=1)

In [57]:
from datasets import Dataset, DatasetDict


esg_dict = {'train' : Dataset.from_pandas(training_pd),
              'eval' : Dataset.from_pandas(training_pd)}

esg_ds = DatasetDict(esg_dict)

# 02 Tokenizing

In [60]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as d

In [61]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_auth_token=True)

In [62]:

def tokenize_function(examples):
    return tokenizer(examples["Sentence"], 
                     padding="max_length", 
                     truncation=True, 
                     max_length=102)

tokenized_datasets = esg_ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/4139 [00:00<?, ? examples/s]

Map:   0%|          | 0/4139 [00:00<?, ? examples/s]

In [63]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4139
    })
    eval: Dataset({
        features: ['Sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4139
    })
})

In [64]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["eval"]

# 03 Finetuning

In [65]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [66]:

from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [67]:

from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset
)

In [68]:

from transformers import TrainingArguments

training_args = TrainingArguments(num_train_epochs=2,
                                  output_dir="output")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [69]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-69-e1ee7db0da93>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [70]:

from transformers import Trainer

trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=full_train_dataset, 
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)
     

In [71]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Sentence. If Sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4139
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1036
  Number of trainable parameters = 108312579


Step,Training Loss
500,0.220900
1000,0.068500


Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1036, training_loss=0.14288267205580782, metrics={'train_runtime': 186.5409, 'train_samples_per_second': 44.376, 'train_steps_per_second': 5.554, 'total_flos': 433908970584264.0, 'train_loss': 0.14288267205580782, 'epoch': 2.0})

In [72]:
import numpy as np

trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Sentence. If Sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4139
  Batch size = 8


{'eval_loss': 0.027286430820822716,
 'eval_accuracy': 0.9939598936941291,
 'eval_runtime': 24.4945,
 'eval_samples_per_second': 168.977,
 'eval_steps_per_second': 21.148,
 'epoch': 2.0}

# 04 Push to HF

In [73]:
model_output = "owen198/esgbert"

In [74]:
# tokenizer.save_pretrained(model_output)
# model.save_pretrained(model_output)

In [75]:
tokenizer.push_to_hub(model_output)
model.push_to_hub(model_output)

tokenizer config file saved in /tmp/tmp36u_6ref/tokenizer_config.json
Special tokens file saved in /tmp/tmp36u_6ref/special_tokens_map.json
Uploading the following files to owen198/esgbert: special_tokens_map.json,vocab.txt,tokenizer_config.json,tokenizer.json
Configuration saved in /tmp/tmp5lvwsh7p/config.json
Model weights saved in /tmp/tmp5lvwsh7p/pytorch_model.bin
Uploading the following files to owen198/esgbert: config.json,pytorch_model.bin


pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/owen198/esgbert/commit/a9ba844bdf6993c54ae1b088d23a5e2a9bc37207', commit_message='Upload BertForSequenceClassification', commit_description='', oid='a9ba844bdf6993c54ae1b088d23a5e2a9bc37207', pr_url=None, pr_revision=None, pr_num=None)

In [78]:
training_pd.tail(30)

,Sentence,label
4109,beware: The product does give the surround so...,2
4110,happy i only wasted money for 2 Music Experie...,2
4111,"oh my goodness!: If this is a single release,...",2
4112,"The dummy ""FATS"" is hysterical!!!!: ALL I can...",2
4113,"Dummy Scared the Be-Jesus Out of Me: Oh, God,...",2
4114,More Ham-O-Rama Theatrics From Sir Anthony: W...,2
4115,Take The Knife Up The Hill And Rent This Movi...,2
4116,MAGIC ADS WERE SCARY!: Though the movie was f...,2
4117,Deliciously disturbing ....Highly Underestima...,2
4118,"Magic: If you like Anthony Hopkins, this is o...",2
